# Regular Genomics Project **RNA Localisation**


---

**Problem definition**:

TODO


### **1.** Data Preparation

Firstly, we import several necessary packages and load in our data:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split

In [2]:
# Initializing test/train split


colab = False  #### Set colab flag ####

if colab:
    np.random.seed(3)
    url = 'https://www.dropbox.com/s/hv4uau8q4wwg00k/final_data.csv?dl=1'
    data_org = pd.read_csv(url)
    test_data = data_org.sample(frac=0.1)
    train_data = data_org.drop(test_data.index)
else:
    np.random.seed(3)
    data_org = pd.read_csv('final_data.csv')
    test_data = data_org.sample(frac=0.1)
    train_data = data_org.drop(test_data.index) # TODO: note: we also have to preprocess the test set similary
    # TODO: colab

# data_org

### **2** |  Initializations

We use as baseline model the [RNATracker](https://github.com/HarveyYan/RNATracker/blob/master/Models/cnn_bilstm_attention.py) model and a CNN model.


In [3]:
max_seq_len = train_data['seq'].apply(lambda x: len(x)).max()

# DataLoader
param_dataLoader_train = {
    'padding_length': max_seq_len,
    "batch_size_train": 32,
    "shuffle_batch_train": True
    }

param_dataLoader_valid = {
    'padding_length': max_seq_len,
    "batch_size_valid": 32,
    "shuffle_batch_valid": True
    }

# KFold
param_KFold = {
    "n_splits": 5,
    "shuffle": True
    }

# Model Parameter
param_model = {
    #'optimizer': 'adam'
    #'loss':
    #'metrics':
}

In [4]:
# Splitting for 5fold

kf = KFold(**param_KFold)
folds = kf.split(train_data)

In [5]:
# training, only need for the model initialization to change in general
from models.CNN import CNN

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []

train_split, valid_split = train_test_split(train_data)

# for i, (train_split, valid_split) in enumerate(folds):
model = CNN(architecure='dcmcdmcmfe',
            dropouts=[.2, .2],
            pooling=[2, 2, 2],
            conv=[{'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'input_shape': (train_data['seq'].apply(lambda x: len(x)).max(), 4)},
                  {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'},
                  {'filters': 64, 'kernel_size': 3, 'activation': 'relu', 'padding': 'same'}],
            dense=[{'units': 9, 'activation': 'softmax'}],
            **param_model )

model.fit(train_data=train_split, **param_dataLoader_train)
results = model.evaluate(eval_data=valid_split, **param_dataLoader_valid)
results = dict(zip(model.model.metrics_names, results))

VALIDATION_ACCURACY.append(results['accuracy'])
VALIDATION_LOSS.append(results['loss'])

# model.model.build( (train_data['seq'].apply(lambda x: len(x)).max(), 4))
# model.model.summary()

2023-06-20 15:53:55.469571: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
test_result = model.evaluate(test_data, **param_dataLoader_valid)
result = dict(zip(model.model.metrics_names, test_result))
TEST_ACCURACY = result['accuracy']
TEST_LOSS = result['loss']

In [21]:
VALIDATION_ACCURACY

[]

In [22]:
VALIDATION_LOSS

[]